<a href="https://colab.research.google.com/github/petermesy/Machine-Learning-Projects/blob/main/winEmbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model on GPU
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device=device)

# File paths
input_path = "/content/amharic_chunks.jsonl"
output_path = "/content/amharic_chunks_with_embeddings.jsonl"

# Process file
with open(input_path, "r", encoding="utf-8") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:

    for line in tqdm(infile, desc="Processing lines"):
        line = line.strip()
        if not line:
            continue

        try:
            data = json.loads(line)
        except json.JSONDecodeError:
            continue

        sentences = data.get("chunked_sentences", [])
        texts = [s.get("sentence", "") for s in sentences]

        # Batch encode with GPU
        embeddings = model.encode(texts, batch_size=32, convert_to_numpy=False, device=device)

        # Add embedding to each sentence
        for s, emb in zip(sentences, embeddings):
            s["embedding"] = emb.tolist()

        data["chunked_sentences"] = sentences
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")


Using device: cuda


Processing lines: 169it [09:17,  3.30s/it]
